In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

### url format for inventory data
https://waterdata.usgs.gov/nwis/inventory/?site_no=07374000&agency_cd=USGS


In [61]:
# url = 'https://waterdata.usgs.gov/nwis/inventory/?site_no=06893000&agency_cd=USGS'
url = 'https://waterdata.usgs.gov/nwis/inventory/?site_no=06186500&agency_cd=USGS'

response = requests.get(url)
response.status_code

200

In [62]:
page = response.text
soup = BeautifulSoup(page,"lxml")

In [63]:
#print(soup.prettify())

In [64]:
station_name = soup.find('h2').text
print(station_name)

USGS 06186500 Yellowstone River at Yellowstone Lk Outlet YNP


In [65]:
description = soup.find(id='stationTable').find_all('dl')[0]
print(description.prettify())

<dl>
 <dt>
  DESCRIPTION:
 </dt>
 <dd>
  Latitude  44°34'01.53",   Longitude 110°22'49.46"   NAD83
  <br/>
 </dd>
 <dd>
  Teton County, Wyoming,  Hydrologic Unit 10070001
 </dd>
 <dd>
  Drainage area: 995 square miles
 </dd>
 <dd>
  Datum of gage:  7,727.77 feet above   NGVD29.
 </dd>
</dl>



In [16]:
drainage = description.find(text=re.compile('Drainage'))
print(drainage)

Drainage area: 1,125,810 square miles


bs4.element.NavigableString

In [17]:
datum = description.find(text=re.compile('Datum'))
print(datum)

Datum of gage:  0.00 feet above   NAVD88.


In [30]:
inventory = soup.find(id='stationTable').find_all('dl')[1]
print(inventory.prettify())

<dl>
 <dt>
  AVAILABLE DATA:
 </dt>
 <dd>
  <table border="1" cellpadding="1" cellspacing="0">
   <tr>
    <th id="data_type">
     Data Type
    </th>
    <th id="begin_date">
     Begin Date
    </th>
    <th id="end_date">
     End Date
    </th>
    <th id="count_nu">
     Count
    </th>
   </tr>
   <tr>
    <td>
     <strong>
      <a href="/nwis/uv?site_no=07374000">
       Current / Historical Observations
      </a>
     </strong>
     (
     <a href="/nwis/?IV_data_availability.html" target="_blank">
      availability statement
     </a>
     )
    </td>
    <td>
     2004-03-17
    </td>
    <td>
     2018-01-25
    </td>
    <td>
    </td>
   </tr>
   <tr>
    <td colspan="4">
     <strong>
      <a href="/nwis/dv?site_no=07374000">
       Daily Data
      </a>
     </strong>
    </td>
   </tr>
   <tr>
    <td headers="data_type">
     Temperature, water, degrees Celsius
    </td>
    <td headers="begin_date">
     2004-10-01
    </td>
    <td headers="end_date">
     2018

In [57]:
discharge = inventory.find(text=re.compile('Discharge'))
print(discharge.strip())
print(discharge.findNext().text.strip())
print(discharge.findNext().findNext().text.strip())
print(discharge.findNext().findNext().findNext().text.strip())

Discharge, cubic feet per second
2004-03-17
2018-01-24
4678


In [42]:
type(discharge)

bs4.element.NavigableString

In [56]:
gage = inventory.find(text=re.compile('Gage'))
print(gage.strip())
print(gage.findNext().text.strip())
print(gage.findNext().findNext().text.strip())
print(gage.findNext().findNext().findNext().text.strip())

Gage height, feet
1997-07-02
2018-01-24
14453


#### <font color='blue'> Pandas can directly read tables from urls. However, it's not garanteed all tables are extracted. Example as blow. The "Inventory" table is found, but the "Description" table is missing. </font>

In [58]:
# use the same url as above for beautifulsoup
url = 'https://waterdata.usgs.gov/nwis/inventory/?site_no=07374000&agency_cd=USGS'

df = pd.read_html(url)[0] # returns a list of tables on that page
df.head(10)

,0,1,2,3
0,Data Type,Begin Date,End Date,Count
1,Current / Historical Observations (availabilit...,2004-03-17,2018-01-25,NaN
2,Daily Data,NaN,NaN,NaN
3,"Temperature, water, degrees Celsius",2004-10-01,2018-01-24,12703
4,"Discharge, cubic feet per second",2004-03-17,2018-01-24,4678
5,"Gage height, feet",1997-07-02,2018-01-24,14453
6,"Specific conductance, water, unfiltered, micro...",2004-10-01,2018-01-24,12270
7,"Dissolved oxygen, water, unfiltered, milligram...",2006-11-04,2018-01-24,3325
8,"pH, water, unfiltered, field, standard units",2004-10-01,2018-01-24,11712
9,"Salinity, water, unfiltered, parts per thousand",2004-10-01,2018-01-24,11973
